In [28]:
import os

# Building a Semantic Search Engine with Python and pgAI

This notebook demonstrates how to build a simple semantic search engine using Python and the pgAI extension for PostgreSQL. We'll use a set of Markdown files as our data source, process them, store them in a database, and then use pgAI's vector search capabilities to perform semantic search.

 **Here's a breakdown of the steps:**

1. **Gather Markdown Files:** We'll start by collecting all the Markdown files from a specified directory.
2. **Chunk the Data:**  Each Markdown file will be split into smaller, more manageable chunks based on headers. This helps in organizing and retrieving information more effectively.
3. **Store in Database:**  We'll store these chunks in a PostgreSQL database, along with relevant metadata.
4. **Embed with pgAI:** Using pgAI's vectorizer, we'll generate embeddings for each chunk of text. These embeddings capture the semantic meaning of the text and are crucial for semantic search.
5. **Semantic Search with SQL:** We'll use SQL queries to perform semantic searches against our database, leveraging the power of pgAI.
6. **Python Integration:** Finally, we'll demonstrate how to integrate these semantic search capabilities into a Python application.


## 1. Gathering Markdown Files

# We'll define a function to locate all Markdown files within a given directory.

In [29]:
def find_markdown_files(directory):
    markdown_files = []
    # Walk through the directory
    for root, dirs, files in os.walk(directory):
        for file in files:
            # Check if the file has a .md extension
            if file.endswith('.md'):
                # Append the full path to the list
                markdown_files.append(os.path.join(root, file))
    return markdown_files

#### Lets quickly print the files to make sure we are good.

In [30]:
# Example usage
directory_path = 'pydantic-ai'
markdown_files = find_markdown_files(directory_path)
print(markdown_files)


['pydantic-ai/README.md', 'pydantic-ai/pydantic_ai_slim/README.md', 'pydantic-ai/tests/example_modules/README.md', 'pydantic-ai/docs/troubleshooting.md', 'pydantic-ai/docs/dependencies.md', 'pydantic-ai/docs/install.md', 'pydantic-ai/docs/help.md', 'pydantic-ai/docs/message-history.md', 'pydantic-ai/docs/testing-evals.md', 'pydantic-ai/docs/multi-agent-applications.md', 'pydantic-ai/docs/results.md', 'pydantic-ai/docs/index.md', 'pydantic-ai/docs/models.md', 'pydantic-ai/docs/contributing.md', 'pydantic-ai/docs/agents.md', 'pydantic-ai/docs/logfire.md', 'pydantic-ai/docs/graph.md', 'pydantic-ai/docs/tools.md', 'pydantic-ai/docs/examples/rag.md', 'pydantic-ai/docs/examples/bank-support.md', 'pydantic-ai/docs/examples/flight-booking.md', 'pydantic-ai/docs/examples/stream-whales.md', 'pydantic-ai/docs/examples/sql-gen.md', 'pydantic-ai/docs/examples/pydantic-model.md', 'pydantic-ai/docs/examples/chat-app.md', 'pydantic-ai/docs/examples/question-graph.md', 'pydantic-ai/docs/examples/index.

In [31]:
# pretty print the markdown file names
for file in markdown_files:
    print(file)

pydantic-ai/README.md
pydantic-ai/pydantic_ai_slim/README.md
pydantic-ai/tests/example_modules/README.md
pydantic-ai/docs/troubleshooting.md
pydantic-ai/docs/dependencies.md
pydantic-ai/docs/install.md
pydantic-ai/docs/help.md
pydantic-ai/docs/message-history.md
pydantic-ai/docs/testing-evals.md
pydantic-ai/docs/multi-agent-applications.md
pydantic-ai/docs/results.md
pydantic-ai/docs/index.md
pydantic-ai/docs/models.md
pydantic-ai/docs/contributing.md
pydantic-ai/docs/agents.md
pydantic-ai/docs/logfire.md
pydantic-ai/docs/graph.md
pydantic-ai/docs/tools.md
pydantic-ai/docs/examples/rag.md
pydantic-ai/docs/examples/bank-support.md
pydantic-ai/docs/examples/flight-booking.md
pydantic-ai/docs/examples/stream-whales.md
pydantic-ai/docs/examples/sql-gen.md
pydantic-ai/docs/examples/pydantic-model.md
pydantic-ai/docs/examples/chat-app.md
pydantic-ai/docs/examples/question-graph.md
pydantic-ai/docs/examples/index.md
pydantic-ai/docs/examples/stream-markdown.md
pydantic-ai/docs/examples/weathe

## 2. Chunking the Markdown Files

To make our data more manageable and improve search accuracy, we'll split each Markdown file into smaller chunks based on headers. This function will handle the chunking process.


In [32]:
#print the contents of the first file
with open(markdown_files[0], 'r') as file:
    print(file.read())

<div align="center">
  <a href="https://ai.pydantic.dev/">
    <picture>
      <source media="(prefers-color-scheme: dark)" srcset="https://ai.pydantic.dev/img/pydantic-ai-dark.svg">
      <img src="https://ai.pydantic.dev/img/pydantic-ai-light.svg" alt="PydanticAI">
    </picture>
  </a>
</div>
<div align="center">
  <em>Agent Framework / shim to use Pydantic with LLMs</em>
</div>
<div align="center">
  <a href="https://github.com/pydantic/pydantic-ai/actions/workflows/ci.yml?query=branch%3Amain"><img src="https://github.com/pydantic/pydantic-ai/actions/workflows/ci.yml/badge.svg?event=push" alt="CI"></a>
  <a href="https://coverage-badge.samuelcolvin.workers.dev/redirect/pydantic/pydantic-ai"><img src="https://coverage-badge.samuelcolvin.workers.dev/pydantic/pydantic-ai.svg" alt="Coverage"></a>
  <a href="https://pypi.python.org/pypi/pydantic-ai"><img src="https://img.shields.io/pypi/v/pydantic-ai.svg" alt="PyPI"></a>
  <a href="https://github.com/pydantic/pydantic-ai"><img src="http

In [33]:
import re
from typing import List, Dict

def chunk_markdown_file(file_path: str) -> List[Dict]:
    with open(file_path, 'r', encoding='utf-8') as file:
        content = file.read()
    
    # Split by headers (## or #)
    chunks = re.split(r'(?=^#{1,2}\s)', content, flags=re.MULTILINE)
    
    processed_chunks = []
    for chunk in chunks:
        if chunk.strip():  # Skip empty chunks
            # Extract header if it exists
            header_match = re.match(r'^#{1,2}\s+(.+)$', chunk.split('\n')[0])
            header = header_match.group(1) if header_match else "No Header"
            
            processed_chunks.append({
                'source_file': file_path,
                'title': header,
                'content': chunk.strip(),
                'metadata': {
                    'file_path': file_path,
                    'section_title': header
                }
            })
    
    return processed_chunks



#### Let's test the chunking function on one of our Markdown files.

In [34]:
# Test on the first markdown file
test_file = markdown_files[10]
print(f"Testing file: {test_file}\n")

# Get chunks for the test file
test_chunks = chunk_markdown_file(test_file)

# Print each chunk in a readable format
for i, chunk in enumerate(test_chunks, 1):
    print(f"\n=== Chunk {i} ===")
    print(f"Title: {chunk['title']}")
    print(f"Source: {chunk['source_file']}")
    print(f"\nContent Preview (first 768 chars):")
    print(chunk['content'][:768], "...\n")
    print("Metadata:", chunk['metadata'])
    print("=" * 50)

Testing file: pydantic-ai/docs/results.md


=== Chunk 1 ===
Title: No Header
Source: pydantic-ai/docs/results.md

Content Preview (first 768 chars):
Results are the final values returned from [running an agent](agents.md#running-agents).
The result values are wrapped in [`RunResult`][pydantic_ai.result.RunResult] and [`StreamedRunResult`][pydantic_ai.result.StreamedRunResult] so you can access other data like [usage][pydantic_ai.usage.Usage] of the run and [message history](message-history.md#accessing-messages-from-results)

Both `RunResult` and `StreamedRunResult` are generic in the data they wrap, so typing information about the data returned by the agent is preserved.

```python {title="olympics.py"}
from pydantic import BaseModel

from pydantic_ai import Agent


class CityLocation(BaseModel):
    city: str
    country: str


agent = Agent('gemini-1.5-flash', result_type=CityLocation)
result = agent. ...

Metadata: {'file_path': 'pydantic-ai/docs/results.md', 'section_title': 'No H

## 3. Storing Chunks in the Database

Now we'll store these chunks in a PostgreSQL database. We'll use the `psycopg2` library to interact with the database. Make sure you have a database set up and replace the connection parameters with your own.


In [35]:
# Process all markdown files
all_chunks = []
for file_path in markdown_files:
    chunks = chunk_markdown_file(file_path)
    all_chunks.extend(chunks)

In [36]:
print(all_chunks)

[{'source_file': 'pydantic-ai/README.md', 'title': 'No Header', 'content': '<div align="center">\n  <a href="https://ai.pydantic.dev/">\n    <picture>\n      <source media="(prefers-color-scheme: dark)" srcset="https://ai.pydantic.dev/img/pydantic-ai-dark.svg">\n      <img src="https://ai.pydantic.dev/img/pydantic-ai-light.svg" alt="PydanticAI">\n    </picture>\n  </a>\n</div>\n<div align="center">\n  <em>Agent Framework / shim to use Pydantic with LLMs</em>\n</div>\n<div align="center">\n  <a href="https://github.com/pydantic/pydantic-ai/actions/workflows/ci.yml?query=branch%3Amain"><img src="https://github.com/pydantic/pydantic-ai/actions/workflows/ci.yml/badge.svg?event=push" alt="CI"></a>\n  <a href="https://coverage-badge.samuelcolvin.workers.dev/redirect/pydantic/pydantic-ai"><img src="https://coverage-badge.samuelcolvin.workers.dev/pydantic/pydantic-ai.svg" alt="Coverage"></a>\n  <a href="https://pypi.python.org/pypi/pydantic-ai"><img src="https://img.shields.io/pypi/v/pydantic-

In [4]:
import psycopg2
import json
from psycopg2.extras import execute_values

In [38]:


# Database connection parameters
db_params = {
    'dbname': 'postgres',
    'user': 'postgres',
    'password': 'postgres',
    'host': 'localhost',
    'port': '5432'
}

# Connect to the database
conn = psycopg2.connect(**db_params)
cur = conn.cursor()

# Prepare the data for insertion
values = [
    (
        chunk['source_file'],
        chunk['title'],
        chunk['content'],
        json.dumps(chunk['metadata'])
    )
    for chunk in all_chunks
]

# Insert the data
execute_values(
    cur,
    """
    INSERT INTO documentation (source_file, title, content, metadata)
    VALUES %s
    """,
    values
)

# Commit and close
conn.commit()
cur.close()
conn.close()

### Lets check the database to make sure our data was added.
- we can improve the chunkign and the storage

In [39]:
# ## Why use PydanticAI

# * __Built by the Pydantic Team__
# Built by the team behind [Pydantic](https://docs.pydantic.dev/latest/) (the validation layer of the OpenAI SDK, the Anthropic SDK, LangChain, LlamaIndex, AutoGPT, Transformers, CrewAI, Instructor and many more).

# * __Model-agnostic__
# Supports OpenAI, Anthropic, Gemini, Ollama, Groq, and Mistral, and there is a simple interface to implement support for [other models](https://ai.pydantic.dev/models/).

# * __Pydantic Logfire Integration__
# Seamlessly [integrates](https://ai.pydantic.dev/logfire/) with [Pydantic Logfire](https://pydantic.dev/logfire) for real-time debugging, performance monitoring, and behavior tracking of your LLM-powered applications.

# * __Type-safe__
# Designed to make [type checking](https://ai.pydantic.dev/agents/#static-type-checking) as powerful and informative as possible for you.

# * __Python-centric Design__
# Leverages Python's familiar control flow and agent composition to build your AI-driven projects, making it easy to apply standard Python best practices you'd use in any other (non-AI) project.

# * __Structured Responses__
# Harnesses the power of [Pydantic](https://docs.pydantic.dev/latest/) to [validate and structure](https://ai.pydantic.dev/results/#structured-result-validation) model outputs, ensuring responses are consistent across runs.

# * __Dependency Injection System__
# Offers an optional [dependency injection](https://ai.pydantic.dev/dependencies/) system to provide data and services to your agent's [system prompts](https://ai.pydantic.dev/agents/#system-prompts), [tools](https://ai.pydantic.dev/tools/) and [result validators](https://ai.pydantic.dev/results/#result-validators-functions).
# This is useful for testing and eval-driven iterative development.

# * __Streamed Responses__
# Provides the ability to [stream](https://ai.pydantic.dev/results/#streamed-results) LLM outputs continuously, with immediate validation, ensuring rapid and accurate results.

# * __Graph Support__
# [Pydantic Graph](https://ai.pydantic.dev/graph) provides a powerful way to define graphs using typing hints, this is useful in complex applications where standard control flow can degrade to spaghetti code.

We'll use pgAI's vectorizer to generate embeddings for each chunk of text. These embeddings capture the semantic meaning of the text and are crucial for semantic search.

**Note:** You'll need to have pgAI installed and configured in your PostgreSQL database. Refer to the pgAI documentation for installation instructions.

**Run the SQL command in your database to create a vectorizer:**

In [40]:

-- Create the vectorizer
SELECT ai.create_vectorizer(
    'documentation'::regclass,
    destination => 'documentation_embeddings',
    embedding => ai.embedding_ollama('nomic-embed-text', 768),
    chunking => ai.chunking_recursive_character_text_splitter('content')
);

SyntaxError: invalid syntax (3111195935.py, line 1)

### Then, create embeddings for the content column in your documentation table:


SELECT
    content,
    embedding <=> ai.ollama_embed('nomic-embed-text', 'what is an agent?', host => 'http://ollama:11434') as distance
FROM documentation_embeddings
ORDER BY distance
LIMIT 5;

## 5. Semantic Search with SQL
Now we can perform semantic searches against our database using SQL queries. pgAI provides the <=> operator for cosine similarity searches. you can find this code as file 04 in the db_utils

-- Semantic Search
SELECT
    content,
    embedding <=> ai.ollama_embed('nomic-embed-text', 'what is an agent?', host => 'http://ollama:11434') as distance
FROM documentation_embeddings
ORDER BY distance
LIMIT 5;

## 6. Python Integration
Finally, let's demonstrate how to integrate these semantic search capabilities into a Python application.

In [5]:
# Database connection parameters
db_params = {
    'dbname': 'postgres',
    'user': 'postgres',
    'password': 'postgres',
    'host': 'localhost',
    'port': '5432'
}

In [6]:
def fetch_search_results(search_text):
    # Reconnect to the database
    conn = psycopg2.connect(**db_params)
    cur = conn.cursor()

    query = """
    SELECT
        content,
        embedding <=> ai.ollama_embed('nomic-embed-text', %s, host => 'http://ollama:11434') as distance
    FROM documentation_embeddings
    ORDER BY distance
    LIMIT 1;
    """

    try:
        # Execute the query with the search_text variable
        cur.execute(query, (search_text,))
        results = cur.fetchall()
        
        # Print results in markdown format
        for row in results:
            print(f"## Search Result (Distance: {row[1]:.4f})\n")
            # add markdown formatting
            print(f"{row[0]}\n")
            print("---\n")

    except Exception as e:
        print(f"An error occurred: {e}")
    
    finally:
        # Always close cursor and connection
        cur.close()
        conn.close()




In [7]:
query = 'how to install pydantic'

fetch_search_results(query)